The ECG criteria for a left bundle branch block include:

- QRS duration greater than 120 milliseconds
- Absence of Q wave in leads I, V5 and V6
- Monomorphic R wave in I, V5 and V6
- ST and T wave displacement opposite to the major deflection of the QRS complex

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

In [ ]:
# clinical set
df_clnc = pd.read_csv('/home/ubuntu/dr-you-ecg-20220420_mount/STEMI_JKL/2023_Jan_testset/230620_clinicaltest(2020).csv')
print(df_clnc.shape)
df_clnc.head()

In [ ]:
dir_clnc = '/home/ubuntu/dr-you-ecg-20220420_mount/STEMI_JKL/2023_Jan_testset/'
X_clnc = np.load(dir_clnc + 'x_clinical_0525.npy')
X_clnc.shape

In [ ]:
# lead 1
X_clnc.shape

In [ ]:
ecg1 = X_clnc[0]
ecg1.shape

In [ ]:
lead1_data = ecg1[:, 0]  # Extract the first lead's data
lead1_data.shape

In [ ]:
'''
Loads an ECG signal with frequency Fs and detects QRS peaks
Author:  Hooman Sedghamiz, Jan, 2019
%% ============== Licensce ========================================== %%
If you use these modules in any other project, please refer to MIT open-source license.
    MIT License
    Copyright (c) 2019 Hooman Sedghamiz
    Permission is hereby granted, free of charge, to any person obtaining a copy
    of this software and associated documentation files (the "Software"), to deal
    in the Software without restriction, including without limitation the rights
    to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
    copies of the Software, and to permit persons to whom the Software is
    furnished to do so, subject to the following conditions:
    The above copyright notice and this permission notice shall be included in all
    copies or substantial portions of the Software.
% THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS 
% "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT 
% LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS 
% FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT
% OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, 
% SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED
% TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR 
% PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF 
% LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING 
% NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS 
% SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
'''
#-------------- Python 3.7 ------------------#
# Lets import loadmat required to import .mat files
from scipy.io import loadmat
# Of course we also need signal from Scipy too
from scipy import signal
# Importing numpy to make it possible to perform vector operations
import numpy as np
# These two libraries are for visualization
import matplotlib.pyplot as plt
from pandas import Series


def BandPassECG(ECG,Fs):
    '''
    This function takes in a "path" and imports the ECG signal in .mat format
    '''
    Fs = 500
    # Implementing the Butterworth BP filter
    W1     = 5*2/Fs                                    # --> 5 Hz cutt-off (high-pass) and Normalize by Sample Rate
    W2     = 15*2/Fs                                   # --> 15 Hz cutt-off (low-pass) and Normalize by Sample Rate
    b, a   = signal.butter(4, [W1,W2], 'bandpass')     # --> create b,a coefficients , since this is IIR we need both b and a coefficients
    ECG    = np.asarray(ECG)                           # --> let's convert the ECG to a numpy array, this makes it possible to perform vector operations 
    ECG    = np.squeeze(ECG)                           # --> squeeze
    ECG_BP = signal.filtfilt(b,a,ECG)    # --> filtering: note we use a filtfilt that compensates for the delay
    return ECG_BP,ECG

def Differentiate(ECG):
    '''
    Compute single difference of the signal ECG
    '''
    ECG_df  = np.diff(ECG)
    ECG_sq  = np.power(ECG_df,2)
    return np.insert(ECG_sq,0, ECG_sq[0])

def MovingAverage(ECG,N=30):
    '''
    Compute moving average of signal ECG with a rectangular window of N
    '''
    window  = np.ones((1,N))/N
    ECG_ma  = np.convolve(np.squeeze(ECG),np.squeeze(window))
    return ECG_ma

def QRSpeaks(ECG,Fs):
    '''
    Finds peaks in a smoothed signal ECG and sampling freq Fs.
    '''
    peaks, _  = signal.find_peaks(ECG, height=np.mean(ECG), distance=round(Fs*0.200))
    return peaks

In [ ]:
# Load and BP the Signal
Fs =500
ECG = lead1_data

# BP Filter
ECG_BP,ECG_raw = BandPassECG(ECG,Fs)

# Difference Filter
ECG_df = Differentiate(ECG_BP)

# Moving Average
ECG_ma = MovingAverage(ECG_df)

# QRS peaks
QRS = QRSpeaks(ECG_ma,Fs)

# Plots
fig = plt.figure(frameon="False") 
plt.plot(np.arange(ECG_raw.shape[0])/Fs,ECG_raw,color='y',label='ECG')
plt.vlines(x=(QRS-15)/Fs,ymin=np.min(ECG_raw),ymax=np.max(ECG_raw),linestyles='dashed',color='r', label='QRS',linewidth=2.0)
plt.ylabel('Amp'); plt.xlabel('Time[S]'); plt.legend()
plt.tight_layout(); plt.show()
plt.rcParams["figure.figsize"] = (40, 8)
#fig.savefig('QRS_pks.png', transparent=True)

In [ ]:
fig = plt.figure(frameon="False") 
#plt.plot(np.arange(ECG_raw.shape[0])/Fs,ECG_raw,color='y',label='ECG')
plt.vlines(x=(QRS-15)/Fs,ymin=np.min(ECG_raw),ymax=np.max(ECG_raw),linestyles='dashed',color='r', label='QRS',linewidth=2.0)
plt.ylabel('Amp'); plt.xlabel('Time[S]'); plt.legend()
plt.tight_layout(); plt.show()

In [ ]:
# 그냥 R peak 임

In [ ]:
# Q, S 찾고싶은거임

In [ ]:
import neurokit2 as nk

# Preprocess ECG signal
signals, info = nk.ecg_process(lead1_data, sampling_rate=500)

# Visualize
nk.ecg_plot(signals)

In [ ]:
signals

Locate P, Q, S and T waves in ECG
-  https://neuropsychology.github.io/NeuroKit/examples/ecg_delineate/ecg_delineate.html

In [ ]:
# Load NeuroKit and other useful packages
import neurokit2 as nk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
ecg_signal = lead1_data

In [ ]:
# Extract R-peaks locations
_, rpeaks = nk.ecg_peaks(ecg_signal, sampling_rate=500)

In [ ]:
# Visualize R-peaks in ECG signal
plot = nk.events_plot(rpeaks['ECG_R_Peaks'], ecg_signal)
plt.rcParams["figure.figsize"] = (30, 6)

In [ ]:
rpeaks

In [ ]:
# Zooming into the first 5 R-peaks
plot = nk.events_plot(rpeaks['ECG_R_Peaks'][:5], ecg_signal[:1800])

In [ ]:
# Delineate the ECG signal
_, waves_peak = nk.ecg_delineate(ecg_signal, rpeaks, sampling_rate=500, method="peak")

In [ ]:
waves_peak.keys()

In [ ]:
# Zooming into the first 3 R-peaks, with focus on T_peaks, P-peaks, Q-peaks and S-peaks
plot = nk.events_plot([waves_peak['ECG_T_Peaks'][:3], 
                       waves_peak['ECG_P_Peaks'][:3],
                       waves_peak['ECG_Q_Peaks'][:3],
                       waves_peak['ECG_S_Peaks'][:3]], ecg_signal[:1000])

In [ ]:
# 안타깝게도 QRS onset, offset이 아닌 Q peak, S peak를 찍는다 ㅠㅠ 

In [ ]:
# Delineate the ECG signal and visualizing all peaks of ECG complexes
_, waves_peak = nk.ecg_delineate(ecg_signal, 
                                 rpeaks, 
                                 sampling_rate=500, 
                                 method="peak", 
                                 show=True, 
                                 show_type='peaks')

In [ ]:
# Delineate the ECG signal
signal_cwt, waves_cwt = nk.ecg_delineate(ecg_signal, 
                                         rpeaks, 
                                         sampling_rate=500, 
                                         method="cwt", 
                                         show=True, 
                                         show_type='all')

In [ ]:
# ECG_R_Onsets ~ ECG_R_Offsets
signal_cwt, waves_cwt = nk.ecg_delineate(ecg_signal, 
                                         rpeaks, 
                                         sampling_rate=500, 
                                         method="cwt", 
                                         show=True, 
                                         show_type='bounds_R')

In [ ]:
signal_cwt

In [ ]:
# 단위 조심 (5000 > 10000ms : *2) 
on = waves_cwt['ECG_R_Onsets']
off = waves_cwt['ECG_R_Offsets']
print(len(on), len(off))

In [ ]:
for i in range(len(ls)):
    a = off[i]-on[i]
    print(a*2)

# 단위 확인: milisecond

In [ ]:
b = []
for i in range(len(ls)):
    a = off[i]-on[i]
    b.append(a*2)

# 단위 확인: milisecond

In [ ]:
b

In [ ]:
average_qrs_interval = np.mean(b)
print(average_qrs_interval)

In [ ]:
def checing_figure(idx):
    print(idx)
    ecg = X_clnc[idx][:,0]
    _, rpeaks = nk.ecg_peaks(ecg, sampling_rate=500)
    signal_cwt, waves_cwt = nk.ecg_delineate(ecg, 
                                             rpeaks, 
                                             sampling_rate=500, 
                                             method="cwt",  
                                             show = True,
                                             show_type='bounds_R')
    on = waves_cwt['ECG_R_Onsets']
    off = waves_cwt['ECG_R_Offsets']

    print(on)
    print(off)

    plot = nk.events_plot(rpeaks['ECG_R_Peaks'], aa)
    plt.rcParams["figure.figsize"] = (30, 6)